<div style=" border-bottom: 8px solid #e3f56c; overflow: hidden; border-radius: 10px; height: 60px; width: 100%; display: flex;">
  <div style="height: 100%; width: 100%; background-color: #3800BB; float: left; text-align: center; display: flex; justify-content: left; align-items: center; font-size: 40px; ">
    <b><span style="color: #FFFFFF; padding: 20px 20px;">XGBoost</span></b>
  </div>
</div>

<div class="alert" style="background-color: #FEDAD5; border-left: 8px solid #B12111; padding: 14px; border-radius: 8px; font-size: 14px; color: #000000;">
  <h5 style="font-size: 20px; margin-bottom: 10px;">
    <strong> Contents </strong>
  </h5>
<hr>
  <p><font size="3" face="Arial" font-size="large">
  <ul type="square">
      
- `DMatrix` for data preprocessing  
- Two main types of model training in XGBoost  
- Important model hyperparameters  
- Hyperparameter tuning  
- `LearningRateScheduler`  
- Custom metrics from `scikit-learn`  
- Conclusions and summary  


  </ul>
  </font></p>


</div>

<div>

<img src='https://cdn-images-1.medium.com/max/800/1*l4PN8hyAO4fMLxUbIxcETA.png' align="left" width=65% height=65% />
</div>

<div class="alert" style="background-color:  #E8F8F5; border-left: 8px solid #1ABC9C; padding: 14px; border-radius: 8px; font-size: 14px; color: #000000;">

`XGBoost` was developed as a research project at the University of Washington. Tianqi Chen and Carlos Guestrin presented their work at the SIGKDD conference in 2016, making a big impact in the machine learning world. Since its release, this algorithm has not only dominated Kaggle competitions but has also powered numerous industrial applications.
<div>

<div class="alert alert-info">

### **Some key features:**
</div>

<div class="alert" style="background-color:  #E8F8F5; border-left: 8px solid #1ABC9C; padding: 14px; border-radius: 8px; font-size: 14px; color: #000000;">

* Rich set of regularization parameters (`lambda` / `gamma` / `alpha`)
* Considered one of the fastest and least resource-intensive boosting algorithms (though debatable)
* Built-in handling of missing values and categorical features (via `one-hot encoding`) (`enable_categorical = True` / `df[cat_col].astype('category')`)
* Native support for metrics from **scikit-learn** as custom metrics
<div>

<div class="alert" style="background-color:  #E8F8F5; border-left: 8px solid #1ABC9C; padding: 14px; border-radius: 8px; font-size: 14px; color: #000000;">

* Native support for `Dask` and `Spark`
* Compatible with popular cloud platforms and supports distributed training across clusters or multiple machines
* Supports most major programming languages
* Integrates well with other frameworks and services (**Optuna**, **Weights & Biases**, etc.)
<div>

<div class="alert" style="background-color:  #E8F8F5; border-left: 8px solid #1ABC9C; padding: 14px; border-radius: 8px; font-size: 14px; color: #000000;">

**Boosting Implementation Details (see the diagram):**

* **Parallelization:** In `XGBoost`, tree construction is parallelized. This is made possible by rearranging the nested loops used to compute split points: the outer loop iterates over tree leaves, and the inner one over features. Swapping the loops allows efficient parallel computation. Initialization occurs during data loading, followed by sorting with parallel threads, which significantly speeds up training.

* **Tree Pruning (`gamma`):** Unlike traditional GBMs that use negative loss reduction as a split-stopping criterion, `XGBoost` employs `max_depth` and performs *post-pruning* based on a minimum loss reduction (`gamma`). This approach improves computational efficiency.

* **Hardware Optimization:** The algorithm is engineered to maximize hardware utilization by creating internal buffers per thread to store gradient statistics.

* **Regularization:** To prevent overfitting, `XGBoost` applies both L1 (LASSO) and L2 (Ridge) regularization.

* **Handling Missing Values:** `XGBoost` handles sparse data gracefully, learning how to assign missing values based on the direction that minimizes loss.

* **Cross-Validation:** The algorithm includes a built-in method for performing cross-validation during training.

</div>

<div class="alert" style="background-color:  #E8F8F5; border-left: 8px solid #1ABC9C; padding: 14px; border-radius: 8px; font-size: 14px; color: #000000;">

From the user’s perspective, `XGBoost` is quite similar to `LightGBM`: it uses a `booster` object, has a dedicated **sklearn API**, supports `dart`, `callbacks`, `scheduler`, etc. However, there are some key differences worth exploring.

</div>

<div class="alert" style="background-color: #FEF9E7; border-left: 8px solid #D4AC0D; padding: 14px; border-radius: 8px; font-size: 14px; color: #000000;">

!pip install xgboost
</div>